# Day 4: Real Dataset, DataLoader, and Training Loop (MNIST)

<b>Main goal today:</b> Learn how to work with real-world datasets using PyTorch’s `Dataset` and `DataLoader`, and build a working digit classifier using the MNIST dataset.

| Concept                 | What it does                                         |
| ----------------------- | ---------------------------------------------------- |
| `torchvision.datasets`  | Load real datasets like MNIST, CIFAR10               |
| `transforms.ToTensor()` | Converts images into tensors                         |
| `DataLoader`            | Automatically batches, shuffles, and loads data      |
| Training loop           | Trains on batches, tracks loss, evaluates            |
| Model saving            | Save and load your trained models using `torch.save` |


#### Step-by-Step: Digit Classification using MNIST

##### 1. Imports and Setup

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

##### 2. Load the MNIST Dataset

In [3]:
# Transform: convert PIL image to tensor (0–255 → 0–1)
transform = transforms.ToTensor()

# Load training and test sets
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)


#### Explanation:

* <b>MNIST:</b> 70,000 grayscale digit images (0–9), 28×28 pixels

* `transform`=transforms.ToTensor():

* Converts image to a 1×28×28 tensor

* Normalizes pixels from 0–255 to 0–1

#### 3. Create DataLoaders

In [4]:
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)

#### Explanation:

* `batch_size=64`: gives 64 images in each training step

* `shuffle=True`: shuffles the training data for better learning

* You’ll loop over these loaders during training/testing

#### 4. Define the Model

In [5]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Flatten(),  # 1×28×28 → 784
            nn.Linear(784,128),
            nn.ReLU(), 
            nn.Linear(128,10)   # 10 output classes (0–9)
        )
    def forward(self,X):
        return self.net(X)

#### Explanation:

* `nn.Flatten():` reshapes 2D image into a flat vector: 28×28 = 784

* `nn.Linear(784, 128):` first hidden layer

* `nn.ReLU():` adds non-linearity

* `nn.Linear(128, 10):` output layer for 10 digit classes

* We wrap everything in `nn.Sequential` for clean and short code

#### 5. Initialize Model, Loss, Optimizer

In [6]:
model=SimpleNN()
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

##### Explanation:

* CrossEntropyLoss is the standard loss for multi-class classification problems.

* Adam is an optimizer like SGD, but faster and smarter.

#### 6. Training Loop

In [7]:
epochs=15

for epoch in range(epochs):
    total_loss=0
    for images,labels in train_loader:
        preds=model(images) #Forward prop
        loss=loss_fn(preds,labels) #Calculate Loss
        
        optimizer.zero_grad() #Clear old gradients
        loss.backward() # BackProp
        optimizer.step() #Update weights
        
        total_loss+=loss.item()
    
    print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 317.9492
Epoch: 2, Loss: 141.6373
Epoch: 3, Loss: 100.3108
Epoch: 4, Loss: 76.7014
Epoch: 5, Loss: 61.0998
Epoch: 6, Loss: 50.6520
Epoch: 7, Loss: 40.4846
Epoch: 8, Loss: 34.0457
Epoch: 9, Loss: 28.1311
Epoch: 10, Loss: 22.8576
Epoch: 11, Loss: 19.0238
Epoch: 12, Loss: 15.9311
Epoch: 13, Loss: 13.7321
Epoch: 14, Loss: 11.4517
Epoch: 15, Loss: 10.0799


#### Explanation:

* `images:` batch of input images (shape: [64, 1, 28, 28])

* `labels:` ground truth digits for each image (0 to 9)

* `model(images):` returns logits (not probabilities) — shape [64, 10]

* `loss.backward():` calculates gradients

* `optimizer.step():` applies gradient updates

* `zero_grad():` clears previous gradients

##### 7. Testing the Model

In [8]:
correct = 0
total = 0

with torch.no_grad():  # Turn off gradient tracking for speed
    for images, labels in test_loader:
        preds = model(images)
        predicted_labels = preds.argmax(dim=1)
        correct += (predicted_labels == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 97.80%


#### Explanation:

* `preds.argmax(dim=1):` gives the class with highest score for each image

* `== labels:` compares prediction with truth

* `sum().item():` counts number of correct predictions

#### 8. Save the model

In [9]:
torch.save(model.state_dict(),'mnist_model.pth')

#### 9. Load Model Later

In [10]:
model=SimpleNN()
model.load_state_dict(torch.load('mnist_model.pth'))

<All keys matched successfully>

| Line                         | Purpose                                   |
| ---------------------------- | ----------------------------------------- |
| `model = SimpleNN()`         | Create an empty model (same structure)    |
| `torch.load(...)`            | Load saved weights from disk              |
| `model.load_state_dict(...)` | Fill the empty model with learned weights |


#### If you don’t call model.eval():

* Dropout layers will still randomly drop units

* BatchNorm will use batch statistics, not global ones

* So your predictions can be unstable or incorrect

⚠️ That’s why you must call model.eval() before using the model for prediction.

In [11]:
#at the time of inference
model.eval()

SimpleNN(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)